In [1]:
import pandas as pd

result = pd.read_csv('cf_report_1272911_aggregated.csv')

len(result), result.columns

(956, Index(['id', 'qual_o_sentimento_predominante_no_texto',
        'qual_o_sentimento_predominante_no_texto:confidence', 'blog_id',
        'blogid', 'comments', 'content', 'datetime', 'img',
        'img_max_polarity_idx', 'img_max_polarity_val', 'labels', 'name',
        'negemo', 'posemo', 'post_id', 'postid', 'relative_polarity', 'title',
        'user_id', 'wc', 'what_is_the_sentiment_feeling_of_the_image',
        'what_is_the_sentiment_feeling_of_the_imageconfidence'],
       dtype='object'))

In [2]:
result_clean = result[result['qual_o_sentimento_predominante_no_texto'] >= 0]
result_clean = result_clean[result_clean['qual_o_sentimento_predominante_no_texto'] <= 2]
result_clean = result_clean[result_clean['what_is_the_sentiment_feeling_of_the_image'] >= 0]
result_clean = result_clean[result_clean['what_is_the_sentiment_feeling_of_the_image'] <= 2]

len(result_clean)

880

In [3]:
liwc_features = pd.read_csv('post_img_polarity_prob.csv.gz')

len(liwc_features), liwc_features.columns

(17826,
 Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'funct', 'pronoun',
        'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'verb',
        'auxverb', 'past', 'present', 'future', 'adverb', 'preps', 'conj',
        'negate', 'quant', 'number', 'swear', 'social', 'family', 'friend',
        'humans', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad',
        'cogmech', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'inhib',
        'incl', 'excl', 'percept', 'see', 'hear', 'feel', 'bio', 'body',
        'health', 'sexual', 'ingest', 'relativ', 'motion', 'space', 'time',
        'work', 'achieve', 'leisure', 'home', 'money', 'relig', 'death',
        'assent', 'nonfl', 'filler', 'post.id', 'blog.id', 'polarity',
        'img_polarity', 'img', 'wc'],
       dtype='object'))

In [4]:
count = 0

result = pd.DataFrame()

for idx in result_clean.index:
    anno = result_clean.loc[idx]
    postid = int(anno['postid'].replace(',','')[:-6])
    feat_idx = liwc_features[liwc_features['post.id'].astype(str).str.contains(str(postid))].index
    feat = liwc_features.loc[feat_idx[0]]
    
    row = pd.concat([anno, feat])
    row_id = len(result)
    for col in row.index:
        result.loc[row_id, col] = row[[col]].values[0]
    result.loc[row_id, 'neg_prob'] = row['img_polarity'].replace('[','').replace(']','').split(' ')[0]
    result.loc[row_id, 'neu_prob'] = row['img_polarity'].replace('[','').replace(']','').split(' ')[1]
    result.loc[row_id, 'pos_prob'] = row['img_polarity'].replace('[','').replace(']','').split(' ')[2]
    
    if len(feat_idx) == 1:
        count += 1
        #break

count

880

In [5]:
result.head(10)

,id,qual_o_sentimento_predominante_no_texto,qual_o_sentimento_predominante_no_texto:confidence,blog_id,blogid,comments,content,datetime,img,img_max_polarity_idx,...,assent,nonfl,filler,post.id,blog.id,polarity,img_polarity,neg_prob,neu_prob,pos_prob
0,1.755476e+09,2.0,0.6182,NaN,"1,63727630922465E+018",1.0,\n\n\nFoto: Priscila Tessarini/Divulgação\n\n\...,NaN,http://1.bp.blogspot.com/-kpnCLrlNLxs/UcDM0Wxr...,2.0,...,0.0,14.0,0.0,5.871793e+18,1.637276e+18,1.0,[0.18435438 0.39385873 0.42178684],0.18435438,0.39385873,0.42178684
1,1.755476e+09,0.0,1.0000,NaN,"1,63745660564182E+018",0.0,Tem hora que bate\nUma tristeza tão grande\nQu...,NaN,http://1.bp.blogspot.com/-3WzdMLu6hyQ/TWLZl8wf...,0.0,...,0.0,7.0,3.0,4.103505e+18,1.637457e+18,11.0,[0.40337726 0.31016406 0.28645867],0.40337726,0.31016406,0.28645867
2,1.755476e+09,2.0,1.0000,NaN,"1,63801247038459E+018",0.0,"Sumi, eu sei que sumi! Mas estou de volta!\nO ...",NaN,http://3.bp.blogspot.com/-kJTxAWTiA3Y/UzOBfq5e...,0.0,...,1.0,1.0,1.0,2.662966e+18,1.638012e+18,4.0,[0.49763256 0.12188172 0.38048568],0.49763256,0.12188172,0.38048568
3,1.755476e+09,2.0,1.0000,NaN,"1,63803781791363E+018",2.0,"Para estar diante de você, para poder te tocar...",NaN,http://2.bp.blogspot.com/_5XWGP61UwlY/TNG0EgaC...,2.0,...,1.0,3.0,0.0,1.112645e+18,1.638038e+18,6.0,[0.18629915 0.35447067 0.45923024],0.18629915,0.35447067,0.45923024
4,1.755476e+09,0.0,0.6760,NaN,"1,61855661882132E+018",0.0,e sabe qual o pior disso tudo ? você vale sim ...,NaN,http://2.bp.blogspot.com/_H0KSpHNMCVU/S4L6BsBv...,2.0,...,1.0,3.0,0.0,8.977511e+18,1.618557e+18,4.0,[0.18232213 0.17588243 0.64179546],0.18232213,0.17588243,0.64179546
5,1.755476e+09,0.0,1.0000,NaN,"1,61913488129519E+018",0.0,"\n\nNão, não,&nbsp;o mundo não me agrada.&nbsp...",NaN,http://3.bp.blogspot.com/-m4vVDgZx7dw/TpSux38z...,0.0,...,0.0,0.0,0.0,5.980208e+17,1.619135e+18,1.0,[0.51946867 0.28575963 0.19477168],0.51946867,0.28575963,0.19477168
6,1.755476e+09,0.0,0.6782,NaN,16418572,0.0,Fonte: David BorNas últimas semana a nacionali...,NaN,http://4.bp.blogspot.com/-AGkUi0UqqH4/Tcbls2yc...,2.0,...,1.0,6.0,0.0,4.842083e+18,1.641857e+07,12.0,[0.14030957 0.42816144 0.43152896],0.14030957,0.42816144,0.43152896
7,1.755476e+09,0.0,0.6987,NaN,"1,60858072610612E+018",0.0,"Recentemente foi inaugurada em Camaçari, Bahia...",NaN,http://3.bp.blogspot.com/_xMa7Q7PVEv0/S-yNdiYi...,0.0,...,0.0,2.0,0.0,4.429436e+18,1.608581e+18,3.0,[0.44355473 0.27522406 0.28122127],0.44355473,0.27522406,0.28122127
8,1.755476e+09,0.0,1.0000,NaN,"1,60858072610612E+018",1.0,Eles ligaram o foda-se. Embora o Comitê Nacion...,NaN,http://4.bp.blogspot.com/_xMa7Q7PVEv0/S-3BBqaq...,2.0,...,0.0,15.0,0.0,7.327778e+18,1.608581e+18,9.0,[0.20877843 0.35712022 0.43410137],0.20877843,0.35712022,0.43410137
9,1.755476e+09,1.0,0.6654,NaN,"1,60104411485479E+018",0.0,Cultura é o termo genérico usado para signific...,NaN,http://1.bp.blogspot.com/_lI-zHOjBOX8/TOKHItl2...,0.0,...,1.0,20.0,0.0,2.941266e+18,1.601044e+18,15.0,[0.51844704 0.22129695 0.26025605],0.51844704,0.22129695,0.26025605


In [6]:
result.to_csv('finalized_dataset.csv.gz', compression='gzip')

In [7]:
columns = ['funct','pronoun','ppron','i','we','you','shehe','they','ipron','article','verb','auxverb','past','present','future','adverb','preps','conj','negate','quant','number','swear','social','family','friend','humans','affect','posemo','negemo','anx','anger','sad','cogmech','insight','cause','discrep','tentat','certain','inhib','incl','excl','percept','see','hear','feel','bio','body','health','sexual','ingest','relativ','motion','space','time','work','achieve','leisure','home','money','relig','death','assent','nonfl','filler']
t4sa_columns = ['neg_prob','neu_prob','pos_prob']

In [8]:
data_t4sa = result[t4sa_columns].values
data_liwc = result[columns].values
columns.extend(t4sa_columns)
data = result[columns].values
target = result['what_is_the_sentiment_feeling_of_the_image'].values

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

classifiers = {
    "Nearest Neighbors": KNeighborsClassifier(3),
    "Linear SVM": LinearSVC( C=0.025),
    "RBF SVM": SVC(gamma=2, C=1),
    "Gaussian Process": GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Neural Net": MLPClassifier(alpha=1),
    "AdaBoost": AdaBoostClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Netowrk": MLPClassifier(hidden_layer_sizes=(82, 40, 10), activation='tanh', solver='adam', max_iter=500),
    "Random Forest": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
}
    
for clf_name, clf in classifiers.items():
    scores = cross_val_score(clf, data, target, cv=10)
    print('T4SA + LIWC:', clf_name, round(np.mean(scores),2))
    
scores = cross_val_score(clf, data_t4sa, target, cv=10)
print('T4SA:', clf_name, round(np.mean(scores),2))

scores = cross_val_score(clf, data_liwc, target, cv=10)
print('LIWC:', clf_name, round(np.mean(scores),2))

acc = accuracy_score(result['img_max_polarity_idx'], result['what_is_the_sentiment_feeling_of_the_image'])
print('T4SA ACC', round(acc,2))
f1 = f1_score(result['img_max_polarity_idx'], result['what_is_the_sentiment_feeling_of_the_image'], average='micro')
print('T4SA F1', round(acc,2))

T4SA + LIWC: Nearest Neighbors 0.41
T4SA + LIWC: Linear SVM 0.54
T4SA + LIWC: RBF SVM 0.52
T4SA + LIWC: Gaussian Process 0.39
T4SA + LIWC: Decision Tree 0.49
T4SA + LIWC: Neural Net 0.48
T4SA + LIWC: AdaBoost 0.51
T4SA + LIWC: Naive Bayes 0.48
T4SA + LIWC: Neural Netowrk 0.5
T4SA + LIWC: Random Forest 0.52
T4SA: Random Forest 0.54
LIWC: Random Forest 0.52
T4SA ACC 0.33
T4SA F1 0.33


In [10]:
result_clean = result[result['qual_o_sentimento_predominante_no_texto'] != 1]
result_clean = result_clean[result_clean['what_is_the_sentiment_feeling_of_the_image'] != 1]

data_t4sa = result_clean[t4sa_columns].values
data_liwc = result_clean[columns].values
columns.extend(t4sa_columns)
data = result_clean[columns].values
target = result_clean['what_is_the_sentiment_feeling_of_the_image'].values

classifiers = {
    "Nearest Neighbors": KNeighborsClassifier(3),
    "Linear SVM": LinearSVC( C=0.025),
    "RBF SVM": SVC(gamma=2, C=1),
    "Gaussian Process": GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Neural Net": MLPClassifier(alpha=1),
    "AdaBoost": AdaBoostClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Netowrk": MLPClassifier(hidden_layer_sizes=(82, 40, 10), activation='tanh', solver='adam', max_iter=500),
    "Random Forest": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
}

for clf_name, clf in classifiers.items():
    scores = cross_val_score(clf, data, target, cv=10)
    print('T4SA + LIWC:', clf_name, round(np.mean(scores),2))
    
scores = cross_val_score(clf, data_t4sa, target, cv=10)
print('T4SA:', clf_name, round(np.mean(scores),2))

scores = cross_val_score(clf, data_liwc, target, cv=10)
print('LIWC:', clf_name, round(np.mean(scores),2))

acc = accuracy_score(result_clean['img_max_polarity_idx'], result_clean['what_is_the_sentiment_feeling_of_the_image'])
print('T4SA ACC', round(acc,2))
f1 = f1_score(result_clean['img_max_polarity_idx'], result_clean['what_is_the_sentiment_feeling_of_the_image'], average='micro')
print('T4SA F1', round(acc,2))

T4SA + LIWC: Nearest Neighbors 0.73
T4SA + LIWC: Linear SVM 0.79
T4SA + LIWC: RBF SVM 0.79
T4SA + LIWC: Gaussian Process 0.57
T4SA + LIWC: Decision Tree 0.78
T4SA + LIWC: Neural Net 0.78
T4SA + LIWC: AdaBoost 0.78
T4SA + LIWC: Naive Bayes 0.73
T4SA + LIWC: Neural Netowrk 0.8
T4SA + LIWC: Random Forest 0.79
T4SA: Random Forest 0.76
LIWC: Random Forest 0.79
T4SA ACC 0.69
T4SA F1 0.69


In [11]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from time import time

# run random search
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

n_iter_search = 200
random_search = RandomizedSearchCV(RandomForestClassifier(n_estimators=20),
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, n_jobs = 4)
start = time()
random_search.fit(data, target)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

random_search.best_params_, random_search.best_score_

RandomizedSearchCV took 22.21 seconds for 200 candidates parameter settings.


({'bootstrap': True,
  'criterion': 'entropy',
  'max_depth': None,
  'max_features': 4,
  'min_samples_leaf': 3,
  'min_samples_split': 2},
 0.8053097345132744)